In [1]:
%load_ext sql

In [2]:
# connection string in SQLAlchemy format
%sql hive://hadoop@localhost:10000/gutenberg

In [3]:
%%sql

Select * from raw_text limit 1

 * hive://hadoop@localhost:10000/gutenberg
Done.


line
The Project Gutenberg eBook of The Magna Carta


In [5]:
%sql Select count(*) from raw_text

 * hive://hadoop@localhost:10000/gutenberg
Done.


_c0
208889834


In [4]:
%%sql 
SELECT explode(split(line,'[ ]')) as word  FROM raw_text LIMIT 5

 * hive://hadoop@localhost:10000/gutenberg
Done.


word
The
Project
Gutenberg
eBook
of


In [5]:
%%sql 
SELECT * FROM (
    SELECT explode(split(line,'[ ]')) as word FROM raw_text
) word
LIMIT 3

 * hive://hadoop@localhost:10000/gutenberg
Done.


word
The
Project
Gutenberg


In [4]:
%%sql 

SELECT word, count(word) as count FROM (
    SELECT explode(split(line,'[ ]')) as word FROM raw_text
) word
GROUP BY word
ORDER BY count DESC
LIMIT 10


 * hive://hadoop@localhost:10000/gutenberg
Done.


word,count
,230055984
the,85373989
of,51678554
and,41021296
to,35040232
a,29558866
in,25832553
that,13802100
was,13139989
is,11569952


## Explain

In [11]:
%%sql 

EXPLAIN EXTENDED SELECT word, count(word) as count FROM (
    SELECT explode(split(line,'[ ]')) as word FROM raw_text
) word
GROUP BY word
ORDER BY count DESC

 * hive://hadoop@localhost:10000/gutenberg
Done.


Explain
STAGE DEPENDENCIES:
Stage-1 is a root stage
Stage-2 depends on stages: Stage-1
Stage-0 depends on stages: Stage-2
""
STAGE PLANS:
Stage: Stage-1
Map Reduce
Map Operator Tree:
TableScan


## creating table

In [ ]:
CREATE TABLE new_test 

AS SELECT word, count(word) as count FROM (
    SELECT explode(split(line,'[ ]')) as word FROM raw_text
) word
GROUP BY word
ORDER BY count DESC